### Testing different features and techniques for better results!!

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline


In [3]:
device = 'gpu' if torch.cuda.is_available() else 'cpu'
device

'gpu'

In [4]:
DATA_PATH = Path("../raw_data/2026-01-13/norm_indicators_BTCUSDT.csv")

TARGET_HORIZON = 1  #predict 1 period ahead
HIDDEN_SIZES=[128,64,32]
DROPOUT = 0.3

BATCH_SIZE = 64
EPOCHS = 100        
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-5

TEST_SIZE = 0.2
VAL_SIZE = 0.1

torch.manual_seed(42)
np.random.seed(42)

In [5]:
data = pd.read_csv(DATA_PATH, index_col=0, parse_dates=True)
data

,RSI_14,MACD_HIST,BB_POSITION,RET_1,RET_5,RET_15,PRICE_EMA21_DIST,ATR,RET_STD,VOL,VWAP_DIST,BODY_ATR,UPPER_WICK_ATR,LOWER_WICK_ATR
Timestamp,,,,,,,,,,,,,,
2021-02-03 08:00:00,0.241043,0.766984,0.743267,-0.377756,0.564120,1.085109,0.886003,-1.229179,0.436301,-0.324545,-0.385126,-0.304051,0.002224,0.424719
2021-02-03 12:00:00,0.331490,0.873115,0.951573,0.780077,0.431135,1.214219,1.406328,-1.195125,0.383037,-0.116924,0.772595,0.814413,0.062215,0.146598
2021-02-03 16:00:00,0.319736,0.813402,0.809429,-0.156423,0.548618,1.047009,1.168740,-1.349722,0.313561,-0.121001,0.440396,-0.058460,0.355312,0.139496
2021-02-03 20:00:00,0.397526,0.948178,0.958863,0.713091,0.456294,1.191007,1.598814,-1.327871,0.260244,-0.461940,1.474987,0.795460,0.041940,0.138442
2021-02-04 00:00:00,0.373060,0.891754,0.817041,-0.184935,0.446622,1.567032,1.296278,-1.335252,0.108684,-0.046030,-0.518228,-0.117380,0.645205,0.131033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-12 12:00:00,0.120618,0.628817,1.060308,1.827616,0.750075,0.185831,0.174942,-0.243553,-0.752349,0.531128,0.743218,1.206470,0.000000,0.569129
2026-01-12 16:00:00,0.153317,0.768563,1.079844,0.295892,0.633060,0.251728,0.292087,-0.019464,-0.781664,1.094078,0.987219,0.218412,0.523808,0.739383
2026-01-12 20:00:00,0.034096,0.653073,0.429864,-1.056101,-0.634100,-0.022114,-0.210253,0.076271,-0.651300,-0.526775,-0.354414,-0.618494,0.202447,0.391847


In [6]:
new = data.drop(columns=['RET_5', 'RET_15'])
new

,RSI_14,MACD_HIST,BB_POSITION,RET_1,PRICE_EMA21_DIST,ATR,RET_STD,VOL,VWAP_DIST,BODY_ATR,UPPER_WICK_ATR,LOWER_WICK_ATR
Timestamp,,,,,,,,,,,,
2021-02-03 08:00:00,0.241043,0.766984,0.743267,-0.377756,0.886003,-1.229179,0.436301,-0.324545,-0.385126,-0.304051,0.002224,0.424719
2021-02-03 12:00:00,0.331490,0.873115,0.951573,0.780077,1.406328,-1.195125,0.383037,-0.116924,0.772595,0.814413,0.062215,0.146598
2021-02-03 16:00:00,0.319736,0.813402,0.809429,-0.156423,1.168740,-1.349722,0.313561,-0.121001,0.440396,-0.058460,0.355312,0.139496
2021-02-03 20:00:00,0.397526,0.948178,0.958863,0.713091,1.598814,-1.327871,0.260244,-0.461940,1.474987,0.795460,0.041940,0.138442
2021-02-04 00:00:00,0.373060,0.891754,0.817041,-0.184935,1.296278,-1.335252,0.108684,-0.046030,-0.518228,-0.117380,0.645205,0.131033
...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-12 12:00:00,0.120618,0.628817,1.060308,1.827616,0.174942,-0.243553,-0.752349,0.531128,0.743218,1.206470,0.000000,0.569129
2026-01-12 16:00:00,0.153317,0.768563,1.079844,0.295892,0.292087,-0.019464,-0.781664,1.094078,0.987219,0.218412,0.523808,0.739383
2026-01-12 20:00:00,0.034096,0.653073,0.429864,-1.056101,-0.210253,0.076271,-0.651300,-0.526775,-0.354414,-0.618494,0.202447,0.391847


In [7]:
new['TARGET_RET'] = new['RET_1'].shift(-1)

In [9]:
new = new.dropna()

In [20]:
# Subset 1: Momentum-focused
features_momentum = ['RSI_14', 'MACD_HIST', 'RET_1']

# Subset 2: Volatility-focused  
features_volatility = ['ATR', 'RET_STD', 'BB_POSITION', 'BODY_ATR']

# Subset 3: Combined best from each
features_hybrid = ['RSI_14', 'ATR', 'VOL', 'VWAP_DIST', 'RET_1']

# Compare directional accuracy of all three

In [15]:
momentum_df

['RSI_14', 'MACD_HIST', 'RET_1']

In [21]:
momentum = new[features_momentum]

In [22]:
momentum

,RSI_14,MACD_HIST,RET_1
Timestamp,,,
2021-02-03 08:00:00,0.241043,0.766984,-0.377756
2021-02-03 12:00:00,0.331490,0.873115,0.780077
2021-02-03 16:00:00,0.319736,0.813402,-0.156423
2021-02-03 20:00:00,0.397526,0.948178,0.713091
2021-02-04 00:00:00,0.373060,0.891754,-0.184935
...,...,...,...
2026-01-12 08:00:00,-0.086680,0.469017,-1.509367
2026-01-12 12:00:00,0.120618,0.628817,1.827616
2026-01-12 16:00:00,0.153317,0.768563,0.295892
